<a href="https://colab.research.google.com/github/MuhammadAbdullah-hash/Ship-image-classifier-using-CNN/blob/master/Computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import zipfile
import cv2
import sklearn.model_selection
from keras import backend as K

from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
with zipfile.ZipFile('/content/train.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/sample_data')

In [3]:
# ##################### Data loading and formating ####################### #
def pre_process(name): 
  data = pd.read_csv(name+'.csv')
  x = np.array(data['image'])
  la = np.array(data['category'])
  new = []
  for i in x:
    y = cv2.imread('/content/images/'+i , 1)
    res = cv2.resize(y , (210,126))
    new.append(res)
  new = np.array(new)
  print(data.info())
  print(data.describe())
  return new , la


w , labels = pre_process('train')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6252 entries, 0 to 6251
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     6252 non-null   object
 1   category  6252 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 97.8+ KB
None
          category
count  6252.000000
mean      2.657550
std       1.528998
min       1.000000
25%       1.000000
50%       2.000000
75%       4.000000
max       5.000000


In [4]:
# #################### Train_test_split #################### #

x_train , x_test , y_train , y_test = sklearn.model_selection.train_test_split(w , labels , test_size=0.1)
print(x_train.shape , y_train.shape)
print(x_test.shape , y_test.shape)
y_dummy = y_test
y_train = to_categorical(y_train)
print(y_train.shape)
y_test = to_categorical(y_test)
print(y_test.shape)

(5626, 126, 210, 3) (5626,)
(626, 126, 210, 3) (626,)
(5626, 6)
(626, 6)


In [0]:
# ################# Model creation ################ #

x_train = x_train/255.0
x_test = x_test/255.0

if K.image_data_format() == 'channels_first':
    input_shape = (3, 126 , 210) 
else:
    input_shape = (126, 210, 3)

model = keras.Sequential([

############## First Convolutional layer so we will specify input_shape here #####################              
keras.layers.Conv2D(64, (3,3), activation='relu', input_shape = input_shape  ),
keras.layers.MaxPooling2D(2, 2),

############## Second Convolutional layer || not specifying input shape #########################
keras.layers.Conv2D(64, (3,3), activation='relu'),
keras.layers.MaxPooling2D(2, 2),

############## Third Convolutional layer || not specifying input shape #########################
keras.layers.Conv2D(64, (3,3), activation='relu'),
keras.layers.MaxPooling2D(2, 2),

############## Fourth Convolutional layer || not specifying input shape ########################
keras.layers.Conv2D(64, (3,3), activation='relu'),
keras.layers.MaxPooling2D(2, 2),


############### Flattening input ################## 
keras.layers.Flatten(),

############## Neurons droped per E-poch ##########
keras.layers.Dropout(0.5),
# 512 neuron hidden layer
keras.layers.Dense(512, activation='relu'),
keras.layers.Dense(6 , activation='softmax')

])

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

# ###### Fitting model ######
model.fit(x_train , y_train , epochs=8 , validation_data=(x_test, y_test))

loss , acc = model.evaluate( x_test , y_test )

print(acc)


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 124, 208, 64)      1792      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 62, 104, 64)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 60, 102, 64)       36928     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 30, 51, 64)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 28, 49, 64)        36928     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 14, 24, 64)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 12, 22, 64)      

In [0]:
# ############### Saving model as hd5 file ################ #

model.save('Ship_Class_2.h5')

In [18]:
# ############ loading the saved model from Keras ################ #

from keras.models import load_model

model = load_model('/content/Ship_Class_2.h5')

print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 124, 208, 64)      1792      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 62, 104, 64)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 60, 102, 64)       36928     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 30, 51, 64)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 28, 49, 64)        36928     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 14, 24, 64)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 12, 22, 64)      

In [0]:
z , lab = pre_process('sample_submission_ns2btKE')
predictions = model.predict(z)

my_dic = {
1:'Cargo' ,
2:'Military',
3:'Carrier',
4:'Cruise',
5:'Tankers', 
}


In [0]:

%matplotlib inline
import matplotlib.pyplot as plt

var = 0
print(my_dic[lab[var]])

plt.imshow(z[var])
plt.xlabel( ' Actuall :  ' + my_dic[lab[var]] )
plt.ylabel( 'Predicted : ' + my_dic[np.argmax(predictions[var])] )

plt.show()
